## Map Mutations to Protein-Drug Interactions
Map and visualize mutations at protein-drug binding sites.

In [1]:
import os
# Disable Numba on Windows: temporary workaround for https://github.com/sbl-sdsc/mmtf-pyspark/issues/288
#os.environ['NUMBA_DISABLE_JIT'] = "1"

In [2]:
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import drugBankDataset, pdbjMineDataset
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd

In [3]:
# Initialize Spark
spark = SparkSession.builder.appName("5-MapToDrugInteractions").getOrCreate()

2022-01-27 12:40:57 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Enable Arrow-based columnar data transfers between Spark and Pandas dataframes
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#### Input parameters

In [5]:
distance_cutoff = 8  # distance cutoff for visualizing interactions
mw_min = 250  # minimum molecular weight for drug molecules
input_file_name = 'mutations3d_ligand.csv'  # mutations mapped to protein-ligand binding sites
output_file_name = 'mutations3d_drug.csv'  # mutations mapped to protein-drug binding sites

## Read 'mutation3d_ligand.csv' file created in the previous step

In [6]:
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str') # PDB residue numbers must be string to handle insertion codes
df['queryGroupNumber'] = df['queryGroupNumber'].astype('str')
df.head()

alignmentId  bitscore chainId  evalue  identity  identityPositive  \
0     17063512   993.416       A     0.0     465.0             466.0   
1     18504755   994.186       A     0.0     465.0             466.0   
2     17063512   993.416       A     0.0     465.0             466.0   
3     18504755   994.186       A     0.0     465.0             466.0   
4     17063513   961.444       A     0.0     450.0             451.0   

                                        midlineAlign  \
0  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
1  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
2  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
3  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
4  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...   

                                            pdbAlign  pdbFrom pdbId  \
0  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
1  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
2  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
3  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
4  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...        1  1opl   

                         ...                          color            var_id  \
0                        ...                          green  9:g.133738358A>T   
1                        ...                          green  9:g.133738358A>T   
2                        ...                          green  9:g.133738358A>T   
3                        ...                          green  9:g.133738358A>T   
4                        ...                          green  9:g.133738358A>T   

   queryGroupId queryChainId queryGroupNumber  targetGroupId targetChainId  \
0           P16            A              702            TYR             A   
1           P16            A              702            TYR             A   
2           P16            A              702            TYR             A   
3           P16            A              702            TYR             A   
4           P16            A              539            TYR             A   

   targetGroupNumber  sequenceIndex  \
0                272            229   
1                272            229   
2                272            229   
3                272            229   
4                272            271   

                                            sequence  
0  MGQQPGKVLGDQRREPQGLSEAARWNSKENLLAGPSENDPNLFVAL...  
1  MGQQPGKVLGDQRREPQGLSEAARWNSKENLLAGPSENDPNLFVAL...  
2  MGQQPGKVLGDQRREPQGLSEAARWNSKENLLAGPSENDPNLFVAL...  
3  MGQQPGKVLGDQRREPQGLSEAARWNSKENLLAGPSENDPNLFVAL...  
4  MGQQPGKVLGDQRRPSLPALHFIKGAGKRDSSRHGGPHCNVFVEHE...  

[5 rows x 49 columns]

## Get InChiKey for ligands in PDB with molecular weight >= 250

In [7]:
ccQuery = "SELECT c.id as ligand_id, c.formula_weight, d.descriptor as inchi_key FROM pdbj.chem_comp c \
           JOIN cc.pdbx_chem_comp_descriptor d ON d.comp_id = c.id \
           WHERE d.type = 'InChIKey' AND c.formula_weight >= " + str(mw_min)

ligands = pdbjMineDataset.get_dataset(ccQuery).dropDuplicates().toPandas()
ligands.head()

ligand_id  formula_weight                    inchi_key
0       PQQ         330.206  MMXZSJMASHPLLR-UHFFFAOYSA-N
1       PY4         334.262  VRMPGTOTVVJQMU-SNVBAGLBSA-N
2       CRT         596.925  VAZQBTJCYODOSV-RISZBRKMSA-N
3       VS4         621.790  PPIYQXGSPPWVLJ-CONSDPRKSA-N
4       107         449.506  MBXKBJLIESPLIK-UHFFFAOYSA-N

## Join dataset on ligand id to add InchiKeys

In [8]:
df = df.merge(ligands, left_on=['queryGroupId'], right_on=['ligand_id'], how='inner')
df = df.drop_duplicates()
df.head()

alignmentId  bitscore chainId  evalue  identity  identityPositive  \
0     17063512   993.416       A     0.0     465.0             466.0   
1     18504755   994.186       A     0.0     465.0             466.0   
4     17063513   961.444       A     0.0     450.0             451.0   
5     18504756   962.214       A     0.0     450.0             451.0   
8     17063549   582.408       B     0.0     271.0             271.0   

                                        midlineAlign  \
0  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
1  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
4  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...   
5  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...   
8  DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...   

                                            pdbAlign  pdbFrom pdbId  \
0  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
1  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
4  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...        1  1opl   
5  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...        1  1opl   
8  DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...        1  2g2h   

              ...              queryChainId  queryGroupNumber  targetGroupId  \
0             ...                         A               702            TYR   
1             ...                         A               702            TYR   
4             ...                         A               539            TYR   
5             ...                         A               539            TYR   
8             ...                         B               532            TYR   

  targetChainId targetGroupNumber  sequenceIndex  \
0             A               272            229   
1             A               272            229   
4             A               272            271   
5             A               272            271   
8             B               272             27   

                                            sequence  ligand_id  \
0  MGQQPGKVLGDQRREPQGLSEAARWNSKENLLAGPSENDPNLFVAL...        P16   
1  MGQQPGKVLGDQRREPQGLSEAARWNSKENLLAGPSENDPNLFVAL...        P16   
4  MGQQPGKVLGDQRRPSLPALHFIKGAGKRDSSRHGGPHCNVFVEHE...        P16   
5  MGQQPGKVLGDQRRPSLPALHFIKGAGKRDSSRHGGPHCNVFVEHE...        P16   
8  GHMSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVK...        P16   

   formula_weight                    inchi_key  
0         427.283  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
1         427.283  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
4         427.283  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
5         427.283  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
8         427.283  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  

[5 rows x 52 columns]

## Download open DrugBank dataset¶

For this demo we use the open DrugBank dataset. One disadvantage of the open DrugBank dataset is that it not only contains approved drugs, but many other compounds in pharmaceutical use such as ethanol, ATP, etc.

In [9]:
drugs = drugBankDataset.get_open_drug_links() \
                       .filter("StandardInChIKey IS NOT NULL") \
                       .toPandas()

DrugBank offers specific datasets for drug groups (ALL, APPROVED, EXPERIMENTAL, NUTRACEUTICAL, ILLLICT, WITHDRAWN, INVESTIGATIONAL) with detailed information (e.g., approval status). To use these datasets, you need to create a free DrugBank account and supply username/passwork in the cell below. 

[Create DrugBank account](https://www.drugbank.ca/public_users/sign_up)

For this demo, we continue with the open drug bank dataset, which has limited information.

In [10]:
# username = "<your DrugBank account username>"
# password = "<your DrugBank account password>"
# drugs = drugBankDataset.get_drug_links("APPROVED", username,password) \
#                        .filter("InChIKey IS NOT NULL") \
#                        .toPandas()
# adjust column names
# drugs = drugs.rename(index=str, columns={"InChIKey": "StandardInChIKey", "Name": "Commonname"})

## Print some sample DrugBank data

In [11]:
drugs.head()

DrugBankID                             AccessionNumbers    Commonname  \
0    DB00006   BIOD00076 | BTD00076 | DB02351 | EXPT03302   Bivalirudin   
1    DB00007                         BIOD00009 | BTD00009    Leuprolide   
2    DB00014                         BIOD00113 | BTD00113     Goserelin   
3    DB00027                         BIOD00036 | BTD00036  Gramicidin D   
4    DB00035  BIOD00061 | BIOD00112 | BTD00061 | BTD00112  Desmopressin   

           CAS        UNII                                           Synonyms  \
0  128270-60-0  TN9BEX005G         Bivalirudin | Bivalirudina | Bivalirudinum   
1   53714-56-0  EFY6W0M8TG  Leuprorelin | Leuprorelina | Leuproreline | Le...   
2   65807-02-5  0F65R8P09N                             Goserelin | Goserelina   
3    1405-97-6  5IE62321P4  Bacillus brevis gramicidin D | Gramicidin | Gr...   
4   16679-58-6  ENR1LLB0FP  1-(3-mercaptopropionic acid)-8-D-arginine-vaso...   

              StandardInChIKey  
0  OIRCOABEOLEUMC-GEJPAHFPSA-N  
1  GFIJNRVAKGFPGQ-LIJARHBVSA-N  
2  BLCLNMBMMGCOAS-URPVMXJPSA-N  
3  NDAYQJDHGXTBJL-MWWSRJDJSA-N  
4  NFLWUMRGJYTJIN-PNIOQBSNSA-N

In [12]:
df = df.merge(drugs, left_on=['inchi_key'], right_on=['StandardInChIKey'], how='inner')
df.head()

alignmentId  bitscore chainId  evalue  identity  identityPositive  \
0     17063512   993.416       A     0.0     465.0             466.0   
1     18504755   994.186       A     0.0     465.0             466.0   
2     17063513   961.444       A     0.0     450.0             451.0   
3     18504756   962.214       A     0.0     450.0             451.0   
4     17063549   582.408       B     0.0     271.0             271.0   

                                        midlineAlign  \
0  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
1  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...   
2  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...   
3  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...   
4  DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...   

                                            pdbAlign  pdbFrom pdbId  \
0  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
1  ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...        1  2fo0   
2  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...        1  1opl   
3  DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...        1  1opl   
4  DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...        1  2g2h   

              ...              ligand_id  formula_weight  \
0             ...                    P16         427.283   
1             ...                    P16         427.283   
2             ...                    P16         427.283   
3             ...                    P16         427.283   
4             ...                    P16         427.283   

                     inchi_key DrugBankID AccessionNumbers  Commonname  \
0  ZIQFYVPVJZEOFS-UHFFFAOYSA-N    DB08339             None   PD-166326   
1  ZIQFYVPVJZEOFS-UHFFFAOYSA-N    DB08339             None   PD-166326   
2  ZIQFYVPVJZEOFS-UHFFFAOYSA-N    DB08339             None   PD-166326   
3  ZIQFYVPVJZEOFS-UHFFFAOYSA-N    DB08339             None   PD-166326   
4  ZIQFYVPVJZEOFS-UHFFFAOYSA-N    DB08339             None   PD-166326   

           CAS        UNII  Synonyms             StandardInChIKey  
0  185039-91-2  0H77F2T4U3      None  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
1  185039-91-2  0H77F2T4U3      None  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
2  185039-91-2  0H77F2T4U3      None  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
3  185039-91-2  0H77F2T4U3      None  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  
4  185039-91-2  0H77F2T4U3      None  ZIQFYVPVJZEOFS-UHFFFAOYSA-N  

[5 rows x 59 columns]

### Keep only unique ligands per structure
Here we drop rows with the same structureId and ligandId.

## Save protein-ligand mapping

In [13]:
df.to_csv(output_file_name, index=False)

## View mutations grouped by protein chain
Use the slider to view each protein chain. Turn labels off for an unobstructed view of the mutations. Interacting ligands are rendered as spheres with green carbon atoms.

In [14]:
chains = df.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['1OPL.A', '2FO0.A', '2G2H.A', '2G2H.B', '2GQG.A', '2GQG.B', '2V7A.A', '2V7A.B', '3QRI.A', '3UE4.A', '4TWP.B', '4WA9.B', '5MO4.A']


## View mutations grouped by protein chain

In [15]:
# Setup viewer
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly, show_surface, show_labels, show_annotations, size, font, i):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
   
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # don't display water molecules
        viewer1.setStyle({'resn': ['HOH','DOD']}, {})
        
        rows = group.shape[0]
        for j in range(0, rows):
            # interacting residue info
            res_num = str(group.iloc[j]['pdbPosition'])
            mod_res = {'resi': res_num, 'chain': chain_id}
            
            # interacting ligand info
            lig_chain = group.iloc[j]['queryChainId']
            lig_num = group.iloc[j]['queryGroupNumber']
            lig_name = df.iloc[i]['Commonname']
            lig_res = {'resi': lig_num, 'chain': lig_chain}
            
            col = group.iloc[j]['color']
            c_col = col + 'Carbon'
            viewer1.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
            viewer1.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})   
            viewer1.addStyle(lig_res, {'sphere': {'colorscheme': 'greenCarbon'}})
            
            annotation = group.iloc[j]['annotation']
            mutation = group.iloc[j]['variationId']
            label = ""
            if show_labels:
                label = label + mutation + " "
            if show_annotations:
                label = label + annotation
            if show_labels or show_annotations:
                viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
                viewer1.addLabel(lig_name, {'fontSize':font}, lig_res)

        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        #print header
        print("PDB Id:", pdb_id, "chain Id:", chain_id, "annotation:", annotation)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + group.iloc[j][a])
                
        return viewer1.show()
       
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, show_annotations=False, size=750, font=8, i=s_widget)

def view_image1():
    return viewer1.png()

In [16]:
view_grouped_mutations(chains);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

## View one mutation at a time
Use the slider to view each mutation. Surrounding residues within the `distance_cutoff` are rendered as orange sticks. Interacting ligands are rendered as spheres with green carbon atoms.

In [17]:
# Setup viewer
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly, show_surface, show_labels, show_annotations, size, font, i):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        global viewer2
        viewer2 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer2.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.7}})
       
        # highlight chain of interest in blue
        viewer2.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'width': 0.6, 'opacity':0.5}})
        
        # non-polymer style
        viewer2.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # don't display water molecules
        viewer2.setStyle({'resn': ['HOH','DOD']}, {})
        
        # interacting residue info
        res_num = str(df.iloc[i]['pdbPosition'])
        mod_res = {'resi': res_num, 'chain': chain_id}
        
        # interacting ligand info
        label = df.iloc[i]['variationId']  
        
        lig_id = df.iloc[i]['queryGroupId']
        lig_chain = df.iloc[i]['queryChainId']
        lig_num = df.iloc[i]['queryGroupNumber']
        lig_name = df.iloc[i]['Commonname']
        lig_res = {'resi': lig_num, 'chain': lig_chain}
        lig_label = lig_id + "-" + lig_chain + lig_num + " " + lig_name
        
        col = df.iloc[i]['color']
        c_col = col + 'Carbon'
        viewer2.addStyle(mod_res, {'sphere':{'color': col, 'opacity': 0.8}})  # TODO opacity has no effect

        annotation = df.iloc[i]['annotation']
        mutation = df.iloc[i]['variationId']
        label = ""
        if show_labels:
            label = label + mutation + " "
        if show_annotations:
            label = label + annotation
        if show_labels or show_annotations:
            viewer2.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, mod_res) 
            viewer2.addLabel(lig_label, {'fontSize':font-2}, lig_res) 
            
        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': distance_cutoff}
        
        # residues surrounding mutation site
        viewer2.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})

        # interacting ligand style
        viewer2.addStyle(lig_res, {'sphere': {'colorscheme': 'greenCarbon'}})   
        
        if show_surface:
             viewer2.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
         
        viewer2.zoomTo(surroundings)
        viewer2.center(surroundings)
        
        # print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "ligand:", lig_id + "-" + lig_chain + lig_num, "mutation:", mutation, "annotation:", annotation)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer2.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, show_annotations=False, size=750, font=12, i=s_widget)

def view_image2():
    return viewer2.png()

In [18]:
view_single_mutation(df, distance_cutoff, 'DrugBankID', 'Commonname');

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [19]:
# Shutdown Spark
spark.stop()